## Importando bibliotecas e ferramentas

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
%sql
USE CATALOG medalhao;
SHOW TABLES IN medalhao.bronze;

database,tableName,isTemporary
bronze,dm_categoria_produtos_traducao,false
bronze,dm_cotacao_dolar,false
bronze,ft_avaliacoes_pedidos,false
bronze,ft_consumidores,false
bronze,ft_geolocalizacao,false
bronze,ft_itens_pedidos,false
bronze,ft_pagamentos_pedidos,false
bronze,ft_pedidos,false
bronze,ft_produtos,false
bronze,ft_vendedores,false


## Tabelas Silver

### ft_consumidores

In [0]:
source_table_consumidores = "bronze.ft_consumidores"
target_table_consumidores = "silver.ft_consumidores"

In [0]:
df_bronze_ft_consumidores = spark.table(source_table_consumidores)
print(f"Count: {df_bronze_ft_consumidores.count()} \n")
display(df_bronze_ft_consumidores.limit(5))
print(df_bronze_ft_consumidores.printSchema())

Count: 99441 



customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,ingestion_timestamp
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,2025-11-12T01:12:31.057Z
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,2025-11-12T01:12:31.057Z
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,2025-11-12T01:12:31.057Z
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,2025-11-12T01:12:31.057Z
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,2025-11-12T01:12:31.057Z


root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


In [0]:
nulos_df_ft_consumidores = df_bronze_ft_consumidores.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_ft_consumidores.columns
]).display()

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,ingestion_timestamp
0,0,0,0,0,0


In [0]:
window_spec_consumidores = (
    Window
    .partitionBy("customer_id")
    .orderBy(F.col("ingestion_timestamp").desc())
)

In [0]:
df_silver_ft_consumidores = (
    df_bronze_ft_consumidores
    .withColumn("rn", F.row_number().over(window_spec_consumidores)) 
    .filter(F.col("rn") == 1)
    .select(
        F.col("customer_id").alias("id_consumidor"),
        F.col("customer_zip_code_prefix").alias("prefixo_cep"),
        F.upper(F.col("customer_city")).alias("cidade"),
        F.upper(F.col("customer_state")).alias("estado")
    )
    .filter(F.col("id_consumidor").isNotNull())
    .withColumn("data_ingestao", F.current_timestamp()) 
)

In [0]:
df_silver_ft_consumidores.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_consumidores)
display(df_silver_ft_consumidores.limit(5))

id_consumidor,prefixo_cep,cidade,estado,data_ingestao
00012a2ce6f8dcda20d059ce98491703,6273,OSASCO,SP,2025-11-13T01:31:29.418Z
000161a058600d5901f007fab4c27140,35550,ITAPECERICA,MG,2025-11-13T01:31:29.418Z
0001fd6190edaaf884bcaf3d49edf079,29830,NOVA VENECIA,ES,2025-11-13T01:31:29.418Z
0002414f95344307404f0ace7a26f1d5,39664,MENDONCA,MG,2025-11-13T01:31:29.418Z
000379cdec625522490c315e70c7a9fb,4841,SAO PAULO,SP,2025-11-13T01:31:29.418Z


### ft_pedidos

In [0]:
source_table_pedidos = "bronze.ft_pedidos"
target_table_pedidos = "silver.ft_pedidos"


In [0]:
mapa_status_data = [
    ("delivered", "entregue"),
    ("invoiced", "faturado"),
    ("shipped", "enviado"),
    ("processing", "em processamento"),
    ("unavailable", "indisponível"),
    ("canceled", "cancelado"),
    ("created", "criado"),
    ("approved", "aprovado")
]
mapa_schema = ["status_ingles", "status_portugues"]
df_mapa_status = spark.createDataFrame(mapa_status_data, schema=mapa_schema)
display(df_mapa_status)

status_ingles,status_portugues
delivered,entregue
invoiced,faturado
shipped,enviado
processing,em processamento
unavailable,indisponível
canceled,cancelado
created,criado
approved,aprovado


In [0]:
df_bronze_ft_pedidos = spark.table(source_table_pedidos)
nulos_df_ft_pedidos = df_bronze_ft_pedidos.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_ft_pedidos.columns
]).display()

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,ingestion_timestamp
0,0,0,0,160,1783,2965,0,0


Não realizamos nada aqui sobre os dados nulos para não perder histórico ou inventar informações.

In [0]:
print(df_bronze_ft_pedidos.count())
print(df_bronze_ft_pedidos.printSchema())

99441
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


In [0]:
df_com_status = df_bronze_ft_pedidos.join(
    df_mapa_status,
    df_bronze_ft_pedidos["order_status"] == df_mapa_status["status_ingles"],
    "left"
)
display(df_com_status.limit(5))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,ingestion_timestamp,status_ingles,status_portugues
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33.000Z,2017-10-02T11:07:15.000Z,2017-10-04T19:55:00.000Z,2017-10-10T21:25:13.000Z,2017-10-18T00:00:00.000Z,2025-11-12T01:12:50.329Z,delivered,entregue
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37.000Z,2018-07-26T03:24:27.000Z,2018-07-26T14:31:00.000Z,2018-08-07T15:27:45.000Z,2018-08-13T00:00:00.000Z,2025-11-12T01:12:50.329Z,delivered,entregue
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49.000Z,2018-08-08T08:55:23.000Z,2018-08-08T13:50:00.000Z,2018-08-17T18:06:29.000Z,2018-09-04T00:00:00.000Z,2025-11-12T01:12:50.329Z,delivered,entregue
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06.000Z,2017-11-18T19:45:59.000Z,2017-11-22T13:39:59.000Z,2017-12-02T00:28:42.000Z,2017-12-15T00:00:00.000Z,2025-11-12T01:12:50.329Z,delivered,entregue
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39.000Z,2018-02-13T22:20:29.000Z,2018-02-14T19:46:34.000Z,2018-02-16T18:17:02.000Z,2018-02-26T00:00:00.000Z,2025-11-12T01:12:50.329Z,delivered,entregue


In [0]:
window_spec_pedido = (
    Window
    .partitionBy("order_id")
    .orderBy(F.col("ingestion_timestamp").desc())
)

In [0]:
df_silver_ft_pedidos = (
        df_com_status
        
        .withColumn("tempo_entrega_dias", F.datediff(
            F.col("order_delivered_customer_date"), 
            F.col("order_purchase_timestamp")       
        ))
        .withColumn("tempo_entrega_estimado_dias", F.datediff(
            F.col("order_estimated_delivery_date"), 
            F.col("order_purchase_timestamp")    
        ))
        .withColumn("diferenca_entrega_dias", 
            F.col("tempo_entrega_dias") - F.col("tempo_entrega_estimado_dias")
        )
        .withColumn("entrega_no_prazo", 
            F.when(F.col("order_delivered_customer_date").isNull(), "Não Entregue")
            .when(F.col("diferenca_entrega_dias") <= 0, "Sim")
            .otherwise("Não")
        )
        .withColumn("rn", F.row_number().over(window_spec_pedido))
        .filter(F.col("rn") == 1)
        .select(
            F.col("order_id").alias("id_pedido"),
            F.col("customer_id").alias("id_consumidor"),
            F.col("status_portugues").alias("status"), 
            F.to_date("order_purchase_timestamp").alias("pedido_compra_timestamp"),
            F.to_date("order_approved_at").alias("pedido_aprovado_timestamp"),
            F.to_date("order_delivered_carrier_date").alias("pedido_carregado_timestamp"),
            F.to_date("order_delivered_customer_date").alias("pedido_entregue_timestamp"),
            F.to_date("order_estimated_delivery_date").alias("pedido_estimativa_entrega_timestamp"),
            
            F.col("tempo_entrega_dias"),
            F.col("tempo_entrega_estimado_dias"),
            F.col("diferenca_entrega_dias"),
            F.col("entrega_no_prazo")
        )
        .filter(F.col("id_pedido").isNotNull())
        .withColumn("data_ingestao", F.current_timestamp())
    )

In [0]:
df_silver_ft_pedidos.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(target_table_pedidos)
display(df_silver_ft_pedidos.limit(5))

id_pedido,id_consumidor,status,pedido_compra_timestamp,pedido_aprovado_timestamp,pedido_carregado_timestamp,pedido_entregue_timestamp,pedido_estimativa_entrega_timestamp,tempo_entrega_dias,tempo_entrega_estimado_dias,diferenca_entrega_dias,entrega_no_prazo,data_ingestao
00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,entregue,2018-08-08,2018-08-08,2018-08-10,2018-08-14,2018-08-20,6,12,-6,Sim,2025-11-13T00:17:20.786Z
00054e8431b9d7675808bcb819fb4a32,32e2e6ab09e778d99bf2e0ecd4898718,entregue,2017-12-10,2017-12-10,2017-12-12,2017-12-18,2018-01-04,8,25,-17,Sim,2025-11-13T00:17:20.786Z
000e562887b1f2006d75e0be9558292e,2b01d668726fb0b751c55918c0437b7b,entregue,2018-02-22,2018-02-22,2018-02-26,2018-03-12,2018-03-19,18,25,-7,Sim,2025-11-13T00:17:20.786Z
00143d0f86d6fbd9f9b38ab440ac16f5,2e45292979b9b2700ea09560eeb0f803,entregue,2017-10-16,2017-10-16,2017-10-16,2017-10-27,2017-11-06,11,21,-10,Sim,2025-11-13T00:17:20.786Z
001c85b5f68d2be0cb0797afc9e8ce9a,48ed31e735f1c420ed6ca3637b7c744d,entregue,2017-11-24,2017-11-24,2017-11-27,2017-12-22,2017-12-14,28,20,8,Não,2025-11-13T00:17:20.786Z


### ft_itens_pedidos

In [0]:
source_table_itens_pedidos = "bronze.ft_itens_pedidos"
target_table_itens_pedidos = "silver.ft_itens_pedidos"

In [0]:
df_bronze_ft_itens_pedidos = spark.table(source_table_itens_pedidos)
nulos_df_ft_itens_pedidos = df_bronze_ft_itens_pedidos.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_ft_itens_pedidos.columns
]).display()

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,ingestion_timestamp
0,0,0,0,0,0,0,0


In [0]:
print(df_bronze_ft_itens_pedidos.count())
print(df_bronze_ft_itens_pedidos.printSchema())

112650
root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


In [0]:
window_spec_itens = (
    Window
    .partitionBy("order_id", "order_item_id")
    .orderBy(F.col("ingestion_timestamp").desc()) 
)

In [0]:
df_silver_ft_itens_pedidos = (
    df_bronze_ft_itens_pedidos 
    .withColumn("rn", F.row_number().over(window_spec_itens))
    .filter(F.col("rn") == 1)
    
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("order_item_id").alias("id_item"),
        F.col("product_id").alias("id_produto"),
        F.col("seller_id").alias("id_vendedor"),
        F.col("price").alias("preco_BRL"),
        F.col("freight_value").alias("preco_frete")
    )
    .filter(F.col("id_pedido").isNotNull())
    .withColumn("data_ingestao", F.current_timestamp()) 
)

In [0]:
df_silver_ft_itens_pedidos.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_itens_pedidos)
display(df_silver_ft_itens_pedidos.limit(5))

id_pedido,id_item,id_produto,id_vendedor,preco_BRL,preco_frete,data_ingestao
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.9,13.29,2025-11-13T00:17:26.262Z
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.9,19.93,2025-11-13T00:17:26.262Z
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.0,17.87,2025-11-13T00:17:26.262Z
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,2025-11-13T00:17:26.262Z
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.9,18.14,2025-11-13T00:17:26.262Z


### ft_pagamentos

In [0]:
source_table_pagamentos = "bronze.ft_pagamentos_pedidos"
target_table_pagamentos = "silver.ft_pagamentos_pedidos"

In [0]:
df_bronze_ft_pagamentos = spark.table(source_table_pagamentos)
nulos_df_ft_pagamentos = df_bronze_ft_pagamentos.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_ft_pagamentos.columns
]).display()

order_id,payment_sequential,payment_type,payment_installments,payment_value,ingestion_timestamp
0,0,0,0,0,0


In [0]:
print(df_bronze_ft_pagamentos.count())
print(df_bronze_ft_pagamentos.printSchema())

103886
root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


In [0]:
window_spec_itens = (
    Window
    .partitionBy("order_id", "payment_sequential")
    .orderBy(F.col("ingestion_timestamp").desc()) 
)

In [0]:
df_silver_ft_pagamentos = (
    df_bronze_ft_pagamentos

    .withColumn("rn", F.row_number().over(window_spec_itens))
    .filter(F.col("rn") == 1)

    .withColumn("forma_pagamento_traduzida",
        F.when(F.col("payment_type") == "credit_card", "Cartão de Crédito")
            .when(F.col("payment_type") == "boleto", "Boleto")
            .when(F.col("payment_type") == "voucher", "Voucher")
            .when(F.col("payment_type") == "debit_card", "Cartão de Débito")
            .otherwise("Outro") 
    )
    
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("payment_sequential").alias("codigo_pagamento"),
        F.col("forma_pagamento_traduzida").alias("forma_pagamento"),
        F.col("payment_installments").alias("parcelas"),
        F.col("payment_value").alias("valor_pagamento")
    )
    .withColumn("data_ingestao", F.current_timestamp())
)

In [0]:
df_silver_ft_pagamentos.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_pagamentos)
display(df_silver_ft_pagamentos.limit(5)) 

id_pedido,codigo_pagamento,forma_pagamento,parcelas,valor_pagamento,data_ingestao
00010242fe8c5a6d1ba2dd792cb16214,1,Cartão de Crédito,2,72.19,2025-11-13T00:17:31.018Z
00018f77f2f0320c557190d7a144bdd3,1,Cartão de Crédito,3,259.83,2025-11-13T00:17:31.018Z
000229ec398224ef6ca0657da4fc703e,1,Cartão de Crédito,5,216.87,2025-11-13T00:17:31.018Z
00024acbcdf0a6daa1e931b038114c75,1,Cartão de Crédito,2,25.78,2025-11-13T00:17:31.018Z
00042b26cf59d7ce69dfabb4e55b4fd9,1,Cartão de Crédito,3,218.04,2025-11-13T00:17:31.018Z


In [0]:
df_check = spark.table(target_table_pagamentos).select("forma_pagamento").distinct()
display(df_check)

forma_pagamento
Cartão de Crédito
Boleto
Voucher
Cartão de Débito
Outro


### ft_avaliacoes_pedidos

In [0]:
source_table_avaliacoes = "bronze.ft_avaliacoes_pedidos"
target_table_avaliacoes = "silver.ft_avaliacoes_pedidos"

In [0]:
df_bronze_ft_avaliacoes = spark.table(source_table_avaliacoes)
nulos_df_ft_avaliacoes = df_bronze_ft_avaliacoes.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_ft_avaliacoes.columns
]).display()

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,ingestion_timestamp
1,2236,2380,92157,63079,8764,8785,0


In [0]:
print(df_bronze_ft_avaliacoes.count())
print(df_bronze_ft_avaliacoes.printSchema())

104162
root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


In [0]:
display(df_bronze_ft_avaliacoes.limit(10))

review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,ingestion_timestamp
7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,null,null,2018-01-18 00:00:00,2018-01-18 21:46:59,2025-11-12T01:12:46.937Z
80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,null,null,2018-03-10 00:00:00,2018-03-11 03:05:13,2025-11-12T01:12:46.937Z
228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,null,null,2018-02-17 00:00:00,2018-02-18 14:36:24,2025-11-12T01:12:46.937Z
e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,null,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,2025-11-12T01:12:46.937Z
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,null,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01 00:00:00,2018-03-02 10:26:53,2025-11-12T01:12:46.937Z
15197aa66ff4d0650b5434f1b46cda19,b18dcdf73be66366873cd26c5724d1dc,1,null,null,2018-04-13 00:00:00,2018-04-16 00:39:37,2025-11-12T01:12:46.937Z
07f9bee5d1b850860defd761afa7ff16,e48aa0d2dcec3a2e87348811bcfdf22b,5,null,null,2017-07-16 00:00:00,2017-07-18 19:30:34,2025-11-12T01:12:46.937Z
7c6400515c67679fbee952a7525281ef,c31a859e34e3adac22f376954e19b39d,5,null,null,2018-08-14 00:00:00,2018-08-14 21:36:06,2025-11-12T01:12:46.937Z
a3f6f7f6f433de0aefbb97da197c554c,9c214ac970e84273583ab523dfafd09b,5,null,null,2017-05-17 00:00:00,2017-05-18 12:05:37,2025-11-12T01:12:46.937Z
8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho,2018-05-22 00:00:00,2018-05-23 16:45:47,2025-11-12T01:12:46.937Z


In [0]:
window_spec_avaliacoes = (
    Window
    .partitionBy("review_id")
    .orderBy(F.col("ingestion_timestamp").desc())
)

In [0]:
df_deduplicado_avaliacoes = df_bronze_ft_avaliacoes.withColumn("rn", F.row_number().over(window_spec_avaliacoes)).filter(F.col("rn") == 1)

count_apos_dedup = df_deduplicado_avaliacoes.count()
linhas_removidas_dedup = df_bronze_ft_avaliacoes.count() - count_apos_dedup
print(f"Linhas removidas por duplicidade: {linhas_removidas_dedup}")
print(f"Linhas restantes após deduplicação: {count_apos_dedup}")

Linhas removidas por duplicidade: 1204
Linhas restantes após deduplicação: 102958


In [0]:
df_com_datas_convertidas = (
    df_deduplicado_avaliacoes
    .withColumn("data_comentario_cast", F.col("review_creation_date").try_cast("timestamp"))
    .withColumn("data_resposta_cast", F.col("review_answer_timestamp").try_cast("timestamp"))
)

print(df_com_datas_convertidas.printSchema())
display(df_com_datas_convertidas.limit(5))

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- rn: integer (nullable = false)
 |-- data_comentario_cast: timestamp (nullable = true)
 |-- data_resposta_cast: timestamp (nullable = true)

None


review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,ingestion_timestamp,rn,data_comentario_cast,data_resposta_cast
null,material de boa qualidade.Agora é amarelo mesmo,pensei que era mais fosco,"mas e muito bonito.eu recomendo""",2018-01-06 00:00:00,2018-01-08 14:20:31,null,2025-11-12T01:12:46.937Z,1,2018-01-08T14:20:31.000Z,null
"""",2018-05-17 00:00:00,2018-05-26 16:45:33,null,null,null,null,2025-11-12T01:12:46.937Z,1,null,null
"06/10/2017""",2017-10-14 00:00:00,2017-10-16 15:21:22,null,null,null,null,2025-11-12T01:12:46.937Z,1,null,null
"A colcha veio junto com os cabides mas não o cominho de mesa """,2018-03-02 00:00:00,2018-03-03 10:03:46,null,null,null,null,2025-11-12T01:12:46.937Z,1,null,null
"Aguardo explicação""",2017-12-03 00:00:00,2017-12-05 23:12:24,null,null,null,null,2025-11-12T01:12:46.937Z,1,null,null


In [0]:
df_validado_avaliacoes = (
    df_com_datas_convertidas
        .filter(F.col("order_id").isNotNull()) #Regra 1: Id de pedido nulo
        .filter(F.col("data_comentario_cast").isNotNull()) # Regra 2: Data Nula
        .filter(F.col("data_comentario_cast") <= F.current_timestamp()) # Regra 3: Data depois da data atual
        .filter(
            (F.col("data_resposta_cast").isNull()) | 
            (F.col("data_resposta_cast") <= F.current_timestamp())
        ) # Regra 4: Data de resposta depois da data atual
)

In [0]:
count_final_silver = df_validado_avaliacoes.count()
linhas_removidas_qualidade = count_apos_dedup - count_final_silver
print(f"Linhas removidas por regras de qualidade: {linhas_removidas_qualidade}")
print(f"Total de linhas prontas para a Silver: {count_final_silver}")

Linhas removidas por regras de qualidade: 8396
Total de linhas prontas para a Silver: 94562


In [0]:
padrao_uuid = "^[0-9a-f]{32}$"
df_filtrado_uuid = (df_validado_avaliacoes
                .filter(F.col("order_id").rlike(padrao_uuid)) #Regra 5: id no padrão uuid
                .filter(F.col("review_id").rlike(padrao_uuid))
)

In [0]:
df_silver_ft_pagamentos = (
    df_filtrado_uuid
    .select(
        F.col("review_id").alias("id_avaliacao"),
        F.col("order_id").alias("id_pedido"),
        F.col("review_score").alias("avaliacao"),
        F.col("review_comment_title").alias("titulo_comentario"),
        F.col("review_comment_message").alias("comentario"),
        F.col("data_comentario_cast").alias("data_comentario"),
        F.col("data_resposta_cast").alias("data_resposta")
    )
    .withColumn("data_ingestao", F.current_timestamp())
)
df_silver_ft_pagamentos.write.mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_avaliacoes)
display(df_silver_ft_pagamentos.limit(5))

id_avaliacao,id_pedido,avaliacao,titulo_comentario,comentario,data_comentario,data_resposta,data_ingestao
0001239bc1de2e33cb583967c2ca4c67,fc046d7776171871436844218f817d7d,5,null,null,2018-03-20T00:00:00.000Z,2018-03-20T18:36:04.000Z,2025-11-13T00:17:40.734Z
0001cc6860aeaf5b9017fe4131a52e62,d4665434b01caa9dc3e3e78b3eb3593e,5,null,null,2018-06-22T00:00:00.000Z,2018-06-26T13:51:29.000Z,2025-11-13T00:17:40.734Z
00020c7512a52e92212f12d3e37513c0,e28abf2eb2f1fbcbdc2dd0cd9a561671,5,Entrega rápida!,A entrega foi super rápida e o pendente é lindo! Igual a foto mesmo!,2018-04-25T00:00:00.000Z,2018-04-26T14:55:36.000Z,2025-11-13T00:17:40.734Z
00032b0141443497c898b3093690af51,04fb47576993a3cb0c12d4b25eab6e4e,5,null,null,2017-05-30T00:00:00.000Z,2017-06-01T23:28:55.000Z,2025-11-13T00:17:40.734Z
00034d88989f9a4c393bdcaec301537f,5f358d797a49fe2f24352f73426215f6,5,null,null,2017-08-12T00:00:00.000Z,2017-08-13T19:56:53.000Z,2025-11-13T00:17:40.734Z


### ft_produtos

In [0]:
source_table_produtos = "bronze.ft_produtos"
target_table_produtos = "silver.ft_produtos"

In [0]:
df_bronze_ft_produtos = spark.table(source_table_produtos)
nulos_df_ft_produtos = df_bronze_ft_produtos.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_ft_produtos.columns
]).display()

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,ingestion_timestamp
0,610,610,610,610,2,2,2,2,0


In [0]:
print(df_bronze_ft_produtos.count())
print(df_bronze_ft_produtos.printSchema())

32951
root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


In [0]:
display(df_bronze_ft_produtos.limit(10))

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,ingestion_timestamp
1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14,2025-11-12T01:12:53.517Z
3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20,2025-11-12T01:12:53.517Z
96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15,2025-11-12T01:12:53.517Z
cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26,2025-11-12T01:12:53.517Z
9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13,2025-11-12T01:12:53.517Z
41d3672d4792049fa1779bb35283ed13,instrumentos_musicais,60,745,1,200,38,5,11,2025-11-12T01:12:53.517Z
732bd381ad09e530fe0a5f457d81becb,cool_stuff,56,1272,4,18350,70,24,44,2025-11-12T01:12:53.517Z
2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,56,184,2,900,40,8,40,2025-11-12T01:12:53.517Z
37cc742be07708b53a98702e77a21a02,eletrodomesticos,57,163,1,400,27,13,17,2025-11-12T01:12:53.517Z
8c92109888e8cdf9d66dc7e463025574,brinquedos,36,1156,1,600,17,10,12,2025-11-12T01:12:53.517Z


In [0]:
window_spec_produtos = (
    Window
    .partitionBy("product_id")
    .orderBy(F.col("ingestion_timestamp").desc())
)

In [0]:
df_silver_ft_produtos = (
    df_bronze_ft_produtos
    .filter(F.col("product_id").rlike(padrao_uuid))
    .withColumn("rn", F.row_number().over(window_spec_produtos))
    .filter(F.col("rn") == 1)

    .select(
        F.col("product_id").alias("id_produto"),
        F.col("product_category_name").alias("categoria_produto"),
        F.col("product_weight_g").alias("peso_produto_gramas"), 
        F.col("product_length_cm").alias("comprimento_centimetros"),
        F.col("product_height_cm").alias("altura_centimetros"),
        F.col("product_width_cm").alias("largura_centimetros")
    )
    .withColumn("data_ingestao", F.current_timestamp())
)
df_silver_ft_produtos.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_produtos)
display(df_silver_ft_produtos.limit(5))

id_produto,categoria_produto,peso_produto_gramas,comprimento_centimetros,altura_centimetros,largura_centimetros,data_ingestao
00066f42aeeb9f3007548bb9d3f33c38,perfumaria,300,20,16,16,2025-11-13T00:25:47.007Z
00088930e925c41fd95ebfe695fd2655,automotivo,1225,55,10,26,2025-11-13T00:25:47.007Z
0009406fd7479715e4bef61dd91f2462,cama_mesa_banho,300,45,15,35,2025-11-13T00:25:47.007Z
000b8f95fcb9e0096488278317764d19,utilidades_domesticas,550,19,24,12,2025-11-13T00:25:47.007Z
000d9be29b5207b54e86aa1b1ac54872,relogios_presentes,250,22,11,15,2025-11-13T00:25:47.007Z


### ft_vendedores

In [0]:
source_table_vendedores = "bronze.ft_vendedores"
target_table_vendedores = "silver.ft_vendedores"

In [0]:
df_bronze_ft_vendedores = spark.table(source_table_vendedores)
nulos_df_ft_vendedores = df_bronze_ft_vendedores.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_ft_vendedores.columns
]).display()

seller_id,seller_zip_code_prefix,seller_city,seller_state,ingestion_timestamp
0,0,0,0,0


In [0]:
print(df_bronze_ft_vendedores.count())
print(df_bronze_ft_vendedores.printSchema())

3095
root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


In [0]:
display(df_bronze_ft_vendedores.limit(10))

seller_id,seller_zip_code_prefix,seller_city,seller_state,ingestion_timestamp
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,2025-11-12T01:12:56.018Z
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,2025-11-12T01:12:56.018Z
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,2025-11-12T01:12:56.018Z
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,2025-11-12T01:12:56.018Z
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,2025-11-12T01:12:56.018Z
c240c4061717ac1806ae6ee72be3533b,20920,rio de janeiro,RJ,2025-11-12T01:12:56.018Z
e49c26c3edfa46d227d5121a6b6e4d37,55325,brejao,PE,2025-11-12T01:12:56.018Z
1b938a7ec6ac5061a66a3766e0e75f90,16304,penapolis,SP,2025-11-12T01:12:56.018Z
768a86e36ad6aae3d03ee3c6433d61df,1529,sao paulo,SP,2025-11-12T01:12:56.018Z
ccc4bbb5f32a6ab2b7066a4130f114e3,80310,curitiba,PR,2025-11-12T01:12:56.018Z


In [0]:
window_spec_seller = (
    Window
    .partitionBy("seller_id")
    .orderBy(F.col("ingestion_timestamp").desc())
)
df_silver_ft_vendedores = (
        df_bronze_ft_vendedores
        .filter(F.col("seller_id").rlike(padrao_uuid))
        .withColumn("rn", F.row_number().over(window_spec_seller))
        .filter(F.col("rn") == 1)
        .select(
            F.col("seller_id").alias("id_vendedor"),
            F.col("seller_zip_code_prefix").alias("prefixo_cep"),
            F.upper(F.col("seller_city")).alias("cidade"),  
            F.upper(F.col("seller_state")).alias("estado")   
        )
        .withColumn("data_ingestao", F.current_timestamp())
)
df_silver_ft_vendedores.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_vendedores)
display(df_silver_ft_vendedores.limit(5))

id_vendedor,prefixo_cep,cidade,estado,data_ingestao
0015a82c2db000af6aaaf3ae2ecb0532,9080,SANTO ANDRE,SP,2025-11-13T00:51:54.936Z
001cca7ae9ae17fb1caed9dfb1094831,29156,CARIACICA,ES,2025-11-13T00:51:54.936Z
001e6ad469a905060d959994f1b41e4f,24754,SAO GONCALO,RJ,2025-11-13T00:51:54.936Z
002100f778ceb8431b7a1020ff7ab48f,14405,FRANCA,SP,2025-11-13T00:51:54.936Z
003554e2dce176b5555353e4f3555ac8,74565,GOIANIA,GO,2025-11-13T00:51:54.936Z


### dm_categoria_produtos_traducao

In [0]:
source_table_categoria_produtos = "bronze.dm_categoria_produtos_traducao"
target_table_categoria_produtos = "silver.dm_categoria_produtos_traducao"

In [0]:
df_bronze_dm_categoria_produtos = spark.table(source_table_categoria_produtos)
nulos_dm_categoria_produtos = df_bronze_dm_categoria_produtos.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_dm_categoria_produtos.columns
]).display()

product_category_name,product_category_name_english,ingestion_timestamp
0,0,0


In [0]:
print(df_bronze_dm_categoria_produtos.count())
print(df_bronze_dm_categoria_produtos.printSchema())
display(df_bronze_dm_categoria_produtos.limit(5))

71
root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


product_category_name,product_category_name_english,ingestion_timestamp
beleza_saude,health_beauty,2025-11-12T01:12:58.312Z
informatica_acessorios,computers_accessories,2025-11-12T01:12:58.312Z
automotivo,auto,2025-11-12T01:12:58.312Z
cama_mesa_banho,bed_bath_table,2025-11-12T01:12:58.312Z
moveis_decoracao,furniture_decor,2025-11-12T01:12:58.312Z


In [0]:
window_spec = (
    Window
    .partitionBy("product_category_name_english")
    .orderBy(F.col("ingestion_timestamp").desc())
)

df_silver_dm_categoria_produtos = (
    df_bronze_dm_categoria_produtos
    .withColumn("rn", F.row_number().over(window_spec))
    .filter(F.col("rn") == 1)
    .select(
        F.col("product_category_name").alias("nome_produto_pt"),
        F.col("product_category_name_english").alias("nome_produto_en")
    )
    .withColumn("data_ingestao", F.current_timestamp())
)
df_silver_dm_categoria_produtos.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_categoria_produtos)
display(df_silver_dm_categoria_produtos.limit(5))

nome_produto_pt,nome_produto_en,data_ingestao
agro_industria_e_comercio,agro_industry_and_commerce,2025-11-13T01:17:03.267Z
climatizacao,air_conditioning,2025-11-13T01:17:03.267Z
artes,art,2025-11-13T01:17:03.267Z
artes_e_artesanato,arts_and_craftmanship,2025-11-13T01:17:03.267Z
audio,audio,2025-11-13T01:17:03.267Z


### dm_cotacao_dolar

In [0]:
source_table_dolar = "bronze.dm_cotacao_dolar"
target_table_dolar = "silver.dm_cotacao_dolar"

In [0]:
window_spec_dolar = (
        Window
        .partitionBy(F.to_date(F.col("dataHoraCotacao")))
        .orderBy(F.col("ingestion_timestamp").desc())
)

In [0]:
df_bronze_dm_cotacao_dolar = spark.table(source_table_dolar)
nulos_dm_cotacao_dolar = df_bronze_dm_cotacao_dolar.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_bronze_dm_cotacao_dolar.columns
]).display()

cotacaoCompra,dataHoraCotacao,ingestion_timestamp
0,0,0


In [0]:
print(df_bronze_dm_cotacao_dolar.count())
print(df_bronze_dm_cotacao_dolar.printSchema())
display(df_bronze_dm_cotacao_dolar.limit(5))

536
root
 |-- cotacaoCompra: double (nullable = true)
 |-- dataHoraCotacao: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

None


cotacaoCompra,dataHoraCotacao,ingestion_timestamp
3.2466,2016-09-01 13:10:10.989,2025-11-13T02:19:26.341Z
3.2425,2016-09-02 13:05:51.688,2025-11-13T02:19:26.341Z
3.2715,2016-09-05 13:09:55.659,2025-11-13T02:19:26.341Z
3.2446,2016-09-06 13:02:39.984,2025-11-13T02:19:26.341Z
3.1928,2016-09-08 13:03:53.968,2025-11-13T02:19:26.341Z


In [0]:
df_bronze_dolar = (
    df_bronze_dm_cotacao_dolar
    .withColumn("rn", F.row_number().over(window_spec_dolar))
    .filter(F.col("rn") == 1)
    .select(
        F.to_date(F.col("dataHoraCotacao")).alias("data_dia"),
        F.col("cotacaoCompra").alias("cotacao_dolar_bronze")
    )
)

In [0]:
min_max_datas = df_bronze_dolar.select(
    F.min("data_dia").alias("data_min"),
    F.max("data_dia").alias("data_max")
).first()

data_min = min_max_datas["data_min"]
data_max = min_max_datas["data_max"]

df_calendario = (
    spark.sql(f"SELECT explode(sequence(to_date('{data_min}'), to_date('{data_max}'), interval 1 day)) AS data_calendario")
)

In [0]:
df_com_nulos = df_calendario.join(
    df_bronze_dolar,
    df_calendario["data_calendario"] == df_bronze_dolar["data_dia"],
    "left"
)

window_spec_locf = (
    Window
    .orderBy("data_calendario")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

df_dolar_full = df_com_nulos.withColumn(
    "cotacao_preenchida",
    F.last("cotacao_dolar_bronze", ignorenulls=True).over(window_spec_locf)
)

In [0]:
df_silver_dm_cotacao_dolar = (
    df_dolar_full
    .select(
        F.col("data_calendario").alias("data"),
        F.col("cotacao_preenchida").alias("cotacao_dolar")
    )
    .filter(F.col("cotacao_dolar").isNotNull()) 
    .withColumn("data_ingestao", F.current_timestamp())
)
df_silver_dm_cotacao_dolar.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table_dolar)
display(df_silver_dm_cotacao_dolar.limit(10))

data,cotacao_dolar,data_ingestao
2016-09-01,3.2466,2025-11-13T02:19:48.175Z
2016-09-02,3.2425,2025-11-13T02:19:48.175Z
2016-09-03,3.2425,2025-11-13T02:19:48.175Z
2016-09-04,3.2425,2025-11-13T02:19:48.175Z
2016-09-05,3.2715,2025-11-13T02:19:48.175Z
2016-09-06,3.2446,2025-11-13T02:19:48.175Z
2016-09-07,3.2446,2025-11-13T02:19:48.175Z
2016-09-08,3.1928,2025-11-13T02:19:48.175Z
2016-09-09,3.2632,2025-11-13T02:19:48.175Z
2016-09-10,3.2632,2025-11-13T02:19:48.175Z


## Integridade Referencial

In [0]:
table_pedidos = "silver.ft_pedidos"
table_consumidores = "silver.ft_consumidores"
table_itens_pedidos = "silver.ft_itens_pedidos"

In [0]:
df_pedidos = spark.table(table_pedidos)
df_consumidores = spark.table(table_consumidores)
df_itens_pedidos = spark.table(table_itens_pedidos)

In [0]:
df_pedidos_orfos = df_pedidos.join(
    df_consumidores,
    on="id_consumidor",
    how="left_anti"  
)
count_pedidos_orfos = df_pedidos_orfos.count()
print(f"Pedidos Órfãos encontrados: {count_pedidos_orfos}")

Pedidos Órfãos encontrados: 0


In [0]:
df_itens_orfos = df_itens_pedidos.join(
    df_pedidos,
    on="id_pedido",
    how="left_anti"  
)
count_itens_orfos = df_itens_orfos.count()
print(f"Itens de Pedido Órfãos encontrados: {count_itens_orfos}")

Itens de Pedido Órfãos encontrados: 0


## Tabela Final

In [0]:
table_pedidos = "silver.ft_pedidos"
table_pagamentos = "silver.ft_pagamentos_pedidos"
table_consumidores = "silver.ft_consumidores"
table_itens_pedidos = "silver.ft_itens_pedidos"
table_vendedores = "silver.ft_vendedores"
table_cotacao = "silver.dm_cotacao_dolar"
target_table = "silver.ft_pedido_total"

In [0]:
df_pagamentos = spark.table(table_pagamentos)

df_pagamentos_agg = (
    df_pagamentos
    .groupBy("id_pedido")
    .agg(
        F.sum("valor_pagamento").alias("valor_total_pago_brl")
    )
)

print("Pagamentos agregados por pedido:")
display(df_pagamentos_agg.limit(5))

Pagamentos agregados por pedido:


id_pedido,valor_total_pago_brl
00010242fe8c5a6d1ba2dd792cb16214,72.19
00018f77f2f0320c557190d7a144bdd3,259.83
000229ec398224ef6ca0657da4fc703e,216.87
00024acbcdf0a6daa1e931b038114c75,25.78
00042b26cf59d7ce69dfabb4e55b4fd9,218.04


In [0]:
df_cotacao = spark.table(table_cotacao)

df_com_pagamento = df_pedidos.join(
    df_pagamentos_agg,
    "id_pedido",
    "left"  
)

df_com_cotacao = df_com_pagamento.join(
    df_cotacao,
    F.to_date(df_com_pagamento["pedido_compra_timestamp"]) == df_cotacao["data"],
    "left" 
)

df_silver_final = (
    df_com_cotacao
    .withColumn(
        "valor_total_pago_brl_tratado",
        F.coalesce(F.col("valor_total_pago_brl"), F.lit(0.0))
    )
    .withColumn(
        "valor_total_pago_usd",
        F.when(
            (F.col("cotacao_dolar").isNotNull()) & (F.col("cotacao_dolar") > 0),
            F.col("valor_total_pago_brl_tratado") / F.col("cotacao_dolar")
        )
        .otherwise(F.lit(None)) 
    )
    
    .select(
        F.col("id_pedido"),
        F.col("id_consumidor"),
        F.col("status"),
        F.col("valor_total_pago_brl_tratado").alias("valor_total_pago_brl"), # Renomeia
        F.round((F.col("valor_total_pago_usd")),2).alias("valor_total_pago_usd"),
        F.to_date(F.col("pedido_compra_timestamp")).alias("data_pedido")
    )
    .withColumn("data_ingestao", F.current_timestamp())

)
df_silver_final.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable(target_table)
display(df_silver_final.limit(15))

id_pedido,id_consumidor,status,valor_total_pago_brl,valor_total_pago_usd,data_pedido,data_ingestao
00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,entregue,72.19,23.04,2017-09-13,2025-11-13T02:23:04.897Z
00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,entregue,259.83,81.6,2017-04-26,2025-11-13T02:23:04.897Z
000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,entregue,216.87,67.37,2018-01-14,2025-11-13T02:23:04.897Z
00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,entregue,25.78,6.87,2018-08-08,2025-11-13T02:23:04.897Z
00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,entregue,218.04,69.81,2017-02-04,2025-11-13T02:23:04.897Z
00048cc3ae777c65dbb7d2a0634bc1ea,816cbea969fe5b689b39cfc97a506742,entregue,34.59,11.16,2017-05-15,2025-11-13T02:23:04.897Z
00054e8431b9d7675808bcb819fb4a32,32e2e6ab09e778d99bf2e0ecd4898718,entregue,31.75,9.68,2017-12-10,2025-11-13T02:23:04.897Z
000576fe39319847cbb9d288c5617fa6,9ed5e522dd9dd85b4af4a077526d8117,entregue,880.75,225.53,2018-07-04,2025-11-13T02:23:04.897Z
0005a1a1728c9d785b8e2b08b904576c,16150771dfd4776261284213b89c304e,entregue,157.6,47.9,2018-03-19,2025-11-13T02:23:04.897Z
0005f50442cb953dcd1d21e1fb923495,351d3cb2cee3c7fd0af6616c82df21d3,entregue,65.39,16.75,2018-07-02,2025-11-13T02:23:04.897Z
